In [1]:
from fastlane_bot import Config, ConfigDB, ConfigNetwork, ConfigProvider
from fastlane_bot.bot import CarbonBot
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC, CPCContainer, T
from fastlane_bot.helpers import TxHelper
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonBot))
from fastlane_bot.testing import *
plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize'] = [12,6]
from fastlane_bot import __VERSION__
require("2.0", __VERSION__)

Using default database url, if you want to use a different database, set the backend_url found at the bottom of manager_base.py
Error adding Ethereum blockchain to database Ethereum, (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "ix_blockchains_name"
DETAIL:  Key (name)=(Ethereum) already exists.

[SQL: INSERT INTO blockchains (name, block_number) VALUES (%(name)s, %(block_number)s) RETURNING blockchains.id]
[parameters: {'name': 'Ethereum', 'block_number': 0}]
(Background on this error at: https://sqlalche.me/e/20/gkpj) skipping...
Error adding exchange carbon_v1 to database, (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "exchanges_pkey"
DETAIL:  Key (id)=(6) already exists.

[SQL: INSERT INTO exchanges (id, name, blockchain_name) VALUES (%(id)s, %(name)s, %(blockchain_name)s)]
[parameters: {'id': 6, 'name': 'carbon_v1', 'blockchain_name': 'Ethereum'}]
(Background on this error at: https://sqlalche.me/e/20/gkpj) ski

C:\Users\nicho\AppData\Local\Temp\ipykernel_58120\1801012949.py:8: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-dark')


Version = 3-b2.1 [requirements >= 2.0 is met]


# Testing the _run functions on TENDERLY [NBTest022]

## MAINNET Configuration

### Set up the bot and curves

In [2]:
C = Config.new(config=Config.CONFIG_TENDERLY)
assert C.DATABASE == C.DATABASE_POSTGRES
# assert C.POSTGRES_DB == "defaultdb"
assert C.NETWORK == C.NETWORK_TENDERLY
assert C.PROVIDER == C.PROVIDER_TENDERLY
bot = CarbonBot(ConfigObj=C)
assert str(type(bot.db)) == "<class 'fastlane_bot.db.manager.DatabaseManager'>"

In [3]:
CCm = bot.get_curves()
exch = {c.P("exchange") for c in CCm}
print("Number of curves:", len(CCm))
print("Number of tokens:", len(CCm.tokens()))
print("Exchanges:", exch)

Number of curves: 3515
Number of tokens: 2230
Exchanges: {'sushiswap_v2', 'uniswap_v3', 'uniswap_v2', 'carbon_v1', 'bancor_v3', 'bancor_v2'}


In [4]:
used_curves = [
    '340282366920938463463374607431768211527-0', #actually deleted from carbon
    '340282366920938463463374607431768211527-1', #actually deleted from carbon
    '2722258935367507707706996859454145691730-1',
    '1701411834604692317316873037158841057360-1',
    '1701411834604692317316873037158841057360-0',
    '9187623906865338513511114400657741709390-0'
]
subcurves = [x for x in CCm if x.cid not in used_curves]
no_0x0 = [x for x in subcurves if '0x0-1AD5' not in x.pair]
CCm2 = CPCContainer([x for x in no_0x0 if x.params.exchange not in ['bancor_v2']])
len(CCm2)

3412

In [5]:
pairs = [x.pair.split('/') for x in CCm if x.params.exchange == 'carbon_v1']
len(set([x for itm in pairs for x in itm]))

23

In [6]:
pairs = [x.pair for x in CCm if x.params.exchange == 'carbon_v1']
len(set(pairs))

50

In [7]:
CCm.bycid("f1112f0481b940058d16a49759a9bb3a").p

28760.523861948794

In [8]:
CCm.bycid("5d5cebf33c0049258c40db6bc90b1558").p

28409.839918723148

In [9]:
# CCm.bycid("9a52c5bdef1c402497d1c4f5d7501f61").p

In [10]:
CCm.bycid("12590447576074723148144860474975423823947-0").p

30899.999999999673

In [11]:
1/CCm.bycid("12590447576074723148144860474975423823947-1").p

31000.001577363997

## Actual run

In [13]:
flt = [T.USDC, T.USDT, T.ETH, T.LINK, T.BNT, T.WBTC, T.DAI]
# bot._run(flashloan_tokens=flt, CCm=CCm2)

## Analytics

In [14]:
ops = bot._find_arbitrage_opportunities_carbon_single_pairwise(flashloan_tokens=flt, CCm=CCm2, result=bot.XS_ARBOPPS)

c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1157: RuntimeWarning: invalid value encountered in subtract
  jac[:, j] = (func(x_plus) - y) / Dxj
c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1157: RuntimeWarning: invalid value encountered in subtract
  jac[:, j] = (func(x_plus) - y) / Dxj
c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: Run

In [15]:
ops

(Decimal('164.497847898955631508793288787357030662879470769149881093085241445805877447128'),
                                                 DAI-1d0F     WBTC-C599
 5d5cebf33c0049258c40db6bc90b1558              945.974947 -3.319780e-02
 12590447576074723148144860474975423823947-0 -1023.181683  3.319780e-02
 AMMIn                                         945.974947  3.319780e-02
 AMMOut                                      -1023.181683 -3.319780e-02
 TOTAL NET                                     -77.206736 -2.108536e-12,
 [{'cid': '5d5cebf33c0049258c40db6bc90b1558',
   'tknin': 'DAI-1d0F',
   'amtin': 945.9749471584801,
   'tknout': 'WBTC-C599',
   'amtout': -0.03319779819131696,
   'error': None},
  {'cid': '12590447576074723148144860474975423823947-0',
   'tknin': 'WBTC-C599',
   'amtin': 0.03319779818920843,
   'tknout': 'DAI-1d0F',
   'amtout': -1023.1816830741591,
   'error': None}],
 'DAI-1d0F',
 (CPCArbOptimizer.TradeInstruction(cid='5d5cebf33c0049258c40db6bc90b1558', tknin='DAI-

In [16]:
# ord = bot._run(flashloan_tokens=flt, CCm=CCm2, result=bot.XS_ORDSCAL)
# ord

In [17]:
# ti = bot._run(flashloan_tokens=flt, CCm=CCm2, result=bot.XS_TI)
# ti

In [18]:
# ti

In [19]:
# ops = bot._find_arbitrage_opportunities(flashloan_tokens=flt, CCm=CCm, result=bot.AO_CANDIDATES)
# ops

In [20]:
# candf = pd.DataFrame(ops).sort_values(by=0, ascending=False)
# candf.reset_index(inplace=True, drop=True)
# candf.iloc[5][1]

In [21]:
# candf.iloc[0][0]

In [22]:
exact = bot._run(flashloan_tokens=flt, CCm=CCm2, result=bot.XS_EXACT)
exact

c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1157: RuntimeWarning: invalid value encountered in subtract
  jac[:, j] = (func(x_plus) - y) / Dxj
c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1157: RuntimeWarning: invalid value encountered in subtract
  jac[:, j] = (func(x_plus) - y) / Dxj
c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: Run

[_calc_carbon_output] before decode:  19186519855587102209396 19186519855587102209396 5856379920129440 7036135960730302
[_calc_carbon_output] after decode:  19186.519855587102209396 19186.519855587102209396 8.45195300565362 167.3320053068149


[TradeInstruction(ConfigObj=Config(network=_ConfigNetworkTenderly(), db=_ConfigDBPostgres(), logger=_ConfigLoggerDefault(), provider=_ConfigProviderTenderly()), cid='5d5cebf33c0049258c40db6bc90b1558', tknin='DAI-1d0F', amtin=Decimal('945.0289722113216'), tknout='WBTC-C599', amtout=Decimal('0.03306550'), pair_sorting='', raw_txs='[]', custom_data='', db=DatabaseManager(ConfigObj=Config(network=_ConfigNetworkTenderly(), db=_ConfigDBPostgres(), logger=_ConfigLoggerDefault(), provider=_ConfigProviderTenderly()), contracts={}, filters=[], data=                                       cid                last_updated  \
 0     00125d26-4f9d-4936-9a46-7e7708cee9b5  2023-05-06 19:04:10.202420   
 1219  5bbd01d2-ba90-49e8-8235-0f5183a44204  2023-05-06 19:11:11.844129   
 1231  5c976f61-5847-4219-a2ee-0b1eb96b2775  2023-05-06 19:18:59.176213   
 1230  5c87da80-15a0-4c59-baac-e8e9661ee920  2023-05-06 19:00:47.931782   
 2635  c21eb1b9-cf04-4ec5-bad1-77fb1b29b38b  2023-05-06 19:13:37.076908   
 ...  

In [23]:
# agg = bot._run(flashloan_tokens=flt, CCm=CCm, result=bot.XS_AGGTI)
# agg

In [24]:
# agg[0].raw_txs

In [25]:
# agg[1].raw_txs

In [26]:
exact[0].amtin_wei, exact[0].amtout_wei

(945028972211321600000, 3306550)

In [27]:
exact[1].amtin_wei, exact[1].amtout_wei

(3306550, 1017076335587934700161)

In [39]:
3286898/3306550

0.9940566451437298

In [28]:
route_struct, flashloan_amount, flashloan_token_address = bot._run(flashloan_tokens=flt, CCm=CCm2, result=bot.XS_ROUTE)

c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1157: RuntimeWarning: invalid value encountered in subtract
  jac[:, j] = (func(x_plus) - y) / Dxj
c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1157: RuntimeWarning: invalid value encountered in subtract
  jac[:, j] = (func(x_plus) - y) / Dxj
c:\Users\nicho\Dropbox\Bancor_Protocol\Github\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: Run

[_calc_carbon_output] before decode:  19186519855587102209396 19186519855587102209396 5856379920129440 7036135960730302
[_calc_carbon_output] after decode:  19186.519855587102209396 19186.519855587102209396 8.45195300565362 167.3320053068149
DAI-1d0F [] True
trade_instruction.cid: 5d5cebf33c0049258c40db6bc90b1558
trade_instruction.cid: 12590447576074723148144860474975423823947


In [29]:
route_struct, flashloan_amount, flashloan_token_address

([{'exchangeId': 4,
   'targetToken': '0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599',
   'minTargetAmount': 1,
   'deadline': 1683693361,
   'customAddress': '0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599',
   'customInt': 3000,
   'customData': '0x'},
  {'exchangeId': 6,
   'targetToken': '0x6B175474E89094C44Da98b954EedeAC495271d0F',
   'minTargetAmount': 1,
   'deadline': 1683693361,
   'customAddress': '0x6B175474E89094C44Da98b954EedeAC495271d0F',
   'customInt': 2000,
   'customData': '0x00000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000250000000000000000000000000000004b0000000000000000000000000000000000000000000000000000000000327436'}],
 945028972211321600000,
 '0x6B175474E89094C44Da98b954EedeAC495271d0F')

# Tenderly Tx Submission

In [30]:
# C.w3.provider.endpoint_uri

In [31]:
# C.w3.eth.getBlock(C.w3.eth.block_number).timestamp

In [32]:
# C.provider.CARBON_CONTROLLER_CONTRACT.strategy(13951577043758477001998358904702496669788)

In [33]:
### This to manually submit
bot._validate_and_submit_transaction_tenderly(
        ConfigObj = C,
        route_struct = route_struct,
        src_amount = flashloan_amount,
        src_address = flashloan_token_address,
            )

{'exchangeId': 4, 'targetToken': '0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599', 'minTargetAmount': 1, 'deadline': 1683693361, 'customAddress': '0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599', 'customInt': 3000, 'customData': '0x'}


{'exchangeId': 6, 'targetToken': '0x6B175474E89094C44Da98b954EedeAC495271d0F', 'minTargetAmount': 1, 'deadline': 1683693361, 'customAddress': '0x6B175474E89094C44Da98b954EedeAC495271d0F', 'customInt': 2000, 'customData': '0x00000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000250000000000000000000000000000004b0000000000000000000000000000000000000000000000000000000000327436'}


Submitting transaction to Tenderly...src_amount=945028972211321600000 src_address=0x6B175474E89094C44Da98b954EedeAC495271d0F


AttributeDict({'transactionHash': HexBytes('0xca65c0306495c0f8a5037040d67b25bcb404cb50aa2f24246b6fe9430fbd1fd0'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x5c4608fddfd0f73edac9ebbe9244f8cc54f3e7e2ebe74be191e63dd0aeb15bec'),
 'blockNumber': 17213260,
 'from': '0x28C6c06298d514Db089934071355E5743bf21d60',
 'to': '0x41Eeba3355d7D6FF628B7982F3F9D055c39488cB',
 'cumulativeGasUsed': 301755,
 'gasUsed': 301755,
 'effectiveGasPrice': 0,
 'contractAddress': None,
 'logs': [],
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

# MAINNNET Preferred Tx Submission Method

In [34]:
# from fastlane_bot.helpers import TxHelpers

In [35]:
# pool = (
#     bot.db.get_pool(exchange_name=bot.ConfigObj.BANCOR_V3_NAME, pair_name='BNT-FF1C/ETH-EEeE')
# )
# bnt_eth = (int(pool.tkn0_balance), int(pool.tkn1_balance))

# # Init TxHelpers
# tx_helpers = TxHelpers(ConfigObj=bot.ConfigObj)
# # Submit tx



In [36]:
# tx_helpers.validate_and_submit_transaction(route_struct=route_struct, src_amt=flashloan_amount,
#                                                     src_address=flashloan_token_address, bnt_eth=bnt_eth,
#                                                     expected_profit=Decimal('1000'), safety_override=True)

# MAINNET Forced Tx Submission Method

In [37]:
# # Initialize tx helper
# tx_helper = TxHelper(
#     usd_gas_limit=55,
#     ConfigObj=bot.ConfigObj,
#     gas_price_multiplier = 1
# )
# # # Submit the transaction
# # tx_helper.submit_flashloan_arb_tx(
# #     arb_data=route_struct,
# #     flashloan_token_address=flashloan_token_address,
# #     flashloan_amount=flashloan_amount,
# #     verbose=True,
# # )

In [38]:
# tx_helper.gas_limit